# Imports and Libraries

In [3]:
from square.client import Client
import os
import pandas as pd

from google_sheets import *
from boto import get_square_secret
import datetime as dt
# import pandas as pd

# import os
import pickle
import uuid

In [4]:
client = Client(
    access_token=get_square_secret()['SQUARE_ACCESS_TOKEN'],
    environment='production')

# Get OTPI Sheet

In [5]:
SAMPLE_SPREADSHEET_ID_input = '1lalOwWx_GwUZgjG3WgyxZvirtT9GYDJ6t3dAuNimTLM'
SAMPLE_RANGE_NAME = 'OTPI Scores!A:V'

opti_scores = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)



# Pick Game and Team to generate invoices for

In [6]:
print(opti_scores['Game Date'].drop_duplicates())

0       5 Aug
63     12 Aug
132    19 Aug
212    26 Aug
279    02 Sep
350    09 Sep
Name: Game Date, dtype: object


In [7]:
game='09 Sep'
teams = ['1','2', '3','4', '5', 'Vets']



invoices_to_raise = opti_scores[opti_scores['Game Date']==game]
invoices_to_raise = invoices_to_raise[invoices_to_raise['OT Team'].isin(teams)]

invoices_to_raise

,Game Date,Competition,Player Name,Position,OT Team,Opposition Name,Result,Goals,Assists,Goals Conceded,...,Red Cards,Clean Sheet?,Penalties Missed,Penalties Saved,Own Goals,MOTM?,Kit,Amount Owed,OPTI Score,Missing
350,09 Sep,League,John Harrison,GK,1,Old Tenisonians,WIN,,,0,...,,TRUE,,,,0,,10,8,FALSE
351,09 Sep,League,Matt Crowe,DEF,1,Old Tenisonians,WIN,,,0,...,,TRUE,,,,0,,10,8,FALSE
352,09 Sep,League,Aaron Snelling,DEF,1,Old Tenisonians,WIN,,,0,...,,TRUE,,,,0,,10,8,FALSE
353,09 Sep,League,Theo Horton,DEF,1,Old Tenisonians,WIN,,,0,...,,TRUE,,,,0,,10,8,FALSE
354,09 Sep,League,Yemi Oviosu,DEF,1,Old Tenisonians,WIN,,1,0,...,,TRUE,,,,0,,10,11,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,09 Sep,Friendly,Rich Lundie,,Vets,Motspur Park Vets,Draw,,,,...,0,TRUE,0,,1,0,,10,0,FALSE
429,09 Sep,Friendly,Jason Raife,,Vets,Motspur Park Vets,Draw,,,,...,0,TRUE,0,,1,0,,10,0,FALSE
430,09 Sep,Friendly,Si Suarez,,Vets,Motspur Park Vets,Draw,,,,...,0,TRUE,0,,1,0,,10,0,FALSE
431,09 Sep,Friendly,Nick Daley,,Vets,Motspur Park Vets,Draw,,,,...,0,TRUE,0,,1,0,,10,0,FALSE


# Get Current Debt

In [8]:
SAMPLE_SPREADSHEET_ID_input = '1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8'
SAMPLE_RANGE_NAME = 'Overall Summary!B2:D1000'

matchday_debt = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)
matchday_debt = matchday_debt[matchday_debt['Name']!='']

matchday_debt

,Name,22-23 Debt,Match Day Debt
0,Aamish Ibrahi,£0.00,£0.00
1,Aaron Snelling,£0.00,£10.00
2,Abdul Adaawe,£0.00,£0.00
3,Abdul Hassan,£0.00,£0.00
4,Adam Platten,£0.00,£0.00
...,...,...,...
310,Will French,£0.00,£5.00
311,Will Salt,£0.00,£0.00
312,Will Sollory,£0.00,£2.50
313,Yemi Oviosu,£0.00,-£70.00


# Merge to Opti

In [9]:
invoice_mg_1 = pd.merge(left=invoices_to_raise, right = matchday_debt, how = 'left', right_on='Name', left_on = 'Player Name')
invoice_mg_1['Match Day Debt'] = pd.to_numeric(invoice_mg_1['Match Day Debt'].str.replace('£', ''))
invoice_mg = invoice_mg_1[invoice_mg_1['Match Day Debt']>0]
#
invoice_mg=invoice_mg[['Player Name', 'OT Team', 'Game Date', 'Opposition Name', 'Amount Owed']]

print('invoices to be made')
display(invoice_mg)
print(invoice_mg.shape)

print('up to date')
display(invoice_mg_1[invoice_mg_1['Match Day Debt']<=0][['Player Name', 'OT Team', 'Game Date', 'Opposition Name', 'Amount Owed']])
print(invoice_mg_1[invoice_mg_1['Match Day Debt']<=0][['Player Name', 'OT Team', 'Game Date', 'Opposition Name', 'Amount Owed']].shape)

invoices to be made


,Player Name,OT Team,Game Date,Opposition Name,Amount Owed
0,John Harrison,1,09 Sep,Old Tenisonians,10
1,Matt Crowe,1,09 Sep,Old Tenisonians,10
2,Aaron Snelling,1,09 Sep,Old Tenisonians,10
3,Theo Horton,1,09 Sep,Old Tenisonians,10
5,Jamie Wood,1,09 Sep,Old Tenisonians,10
6,Matthew Thomas,1,09 Sep,Old Tenisonians,10
7,Stanley Timms,1,09 Sep,Old Tenisonians,5
8,Keaton Cohen,1,09 Sep,Old Tenisonians,5
10,Jarvis Cheek,1,09 Sep,Old Tenisonians,0
11,Ben Dockerill,1,09 Sep,Old Tenisonians,5


(49, 5)
up to date


,Player Name,OT Team,Game Date,Opposition Name,Amount Owed
4,Yemi Oviosu,1,09 Sep,Old Tenisonians,10
9,James Rodriguez,1,09 Sep,Old Tenisonians,10
14,Nick Chimonides,2,09 Sep,Old Tenisonians 2s,10
16,Seb Curle,2,09 Sep,Old Tenisonians 2s,10
19,Cameron Ward,2,09 Sep,Old Tenisonians 2s,5
22,Harry Calvesbert,2,09 Sep,Old Tenisonians 2s,5
23,Finn Coolen,2,09 Sep,Old Tenisonians 2s,5
24,Sean Staunton,2,09 Sep,Old Tenisonians 2s,
30,Jonny Simmons,3,09 Sep,Tiffinians,10
41,Hallam Wye,3,09 Sep,Tiffinians,10


(34, 5)


# Get Customer IDs 

In [10]:
customers_api = client.customers

In [11]:

result = customers_api.list_customers(
    
)
customers = {str(cust.get('given_name')) + ' ' + str(cust.get('family_name')):cust.get('id') for cust in result.body['customers']}


i=1
while result.cursor:
    print(i)
    i+=1
    result = customers_api.list_customers(
        cursor = result.cursor
    )
    customers2 = {str(cust.get('given_name')) + ' ' + str(cust.get('family_name')):cust.get('id') for cust in result.body['customers']}

    customers.update(customers2)




1
2
3
4
5
6
7
8


In [12]:
customer_df=pd.DataFrame(data=[{'name':cust, 'id':customers[cust]} for cust in customers])

In [13]:
id_mg = pd.merge(left = invoice_mg, right = customer_df, how= 'left', left_on = 'Player Name', right_on = 'name')

id_mg['Amount Owed']=pd.to_numeric(id_mg['Amount Owed'])



id_mg

,Player Name,OT Team,Game Date,Opposition Name,Amount Owed,name,id
0,John Harrison,1,09 Sep,Old Tenisonians,10.0,John Harrison,RERYVQH9HD433CNQXNWRVSX78R
1,Matt Crowe,1,09 Sep,Old Tenisonians,10.0,Matt Crowe,7CWJZ4547H035FCMGHFJATYPXW
2,Aaron Snelling,1,09 Sep,Old Tenisonians,10.0,Aaron Snelling,967KF0TYHWZ714NGF440BENAZR
3,Theo Horton,1,09 Sep,Old Tenisonians,10.0,Theo Horton,DV15NAH2F93BF55AMNSDVHCB20
4,Jamie Wood,1,09 Sep,Old Tenisonians,10.0,Jamie Wood,XG84PQRK9D4BH7FD3WNM6R818G
5,Matthew Thomas,1,09 Sep,Old Tenisonians,10.0,Matthew Thomas,9KYM54GAYX1SD5MJAF992YF310
6,Stanley Timms,1,09 Sep,Old Tenisonians,5.0,Stanley Timms,S821CGM0J13R98H0H32VER9TWC
7,Keaton Cohen,1,09 Sep,Old Tenisonians,5.0,Keaton Cohen,YZ989HEH5S0YK90RW4J3T6DTC8
8,Jarvis Cheek,1,09 Sep,Old Tenisonians,0.0,Jarvis Cheek,BYJH2ZRDSH122PJHGD2CWW0ZRR
9,Ben Dockerill,1,09 Sep,Old Tenisonians,5.0,Ben Dockerill,VTRTWFVWSN6JQ9WWWG6NQB6JK0


# Create Orders

In [141]:
orders_api = client.orders

new_orders = []

for i, row in id_mg.iterrows():
    print(i)
    amount = row['Amount Owed'] 
    body = {
        'order': {
            'location_id': 'SBBJSGR24YHYB',
            'reference_id': f'{uuid.uuid1()}',
            'line_items': [
                
                {
                    'quantity': '1',
                    'catalog_object_id': 'OKAX666RRB7XKNM6BJTRM6IM',
                    'base_price_money': {
                        'amount': amount*100,
                        'currency': 'GBP'
                    }
                }
            ]
        },
        'idempotency_key': f'{uuid.uuid1()}'
    }
    # print(body)
    
    result = orders_api.create_order(body)
    # print(result)
    obj_id=result.body['order']['id']

    new_orders.append({'Player Name':row['Player Name'], 'obj_id':obj_id})
    # # print(result)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [142]:
order_mg= pd.merge(left = id_mg, right=pd.DataFrame(new_orders), how= 'left', on = 'Player Name')

order_mg

,Player Name,OT Team,Game Date,Opposition Name,Amount Owed,name,id,obj_id
0,Ade Lawal,4,02 Sep,Suttonians 4s,10,Ade Lawal,T57NBJNMRHX79ZV68VW8SK166R,tfIQU8O9eVkQgGxvtGhFHnNUY7eZY
1,Leuman Coronado,4,02 Sep,Suttonians 4s,10,Leuman Coronado,K97T0162XDDT1NMPN9YQGCH2WR,9vE8KHYhvsMySMb1UPDxwTCfJHPZY
2,Christian King,4,02 Sep,Suttonians 4s,10,Christian King,S7CZGD28C50J12XGC8AWTFG83M,nIiYZ3kfEB2Rp0H1QtoHOqTjqYXZY
3,Joey Wackrill,2,02 Sep,Old Tenisonians 2s,10,Joey Wackrill,VHEZQ85XHX1GK1ZGPW49PR99Z0,hfVxc5adN9bZPCurJJrAWEVCYWZZY
4,Finn Coolen,2,02 Sep,Old Tenisonians 2s,5,Finn Coolen,GX8ZN9VGKTHBW86T2VBBTKZNNM,1ZZN3XvvPDDqWKr5o5OqGUiTT8SZY
5,Sam Laidler,2,02 Sep,Old Tenisonians 2s,5,Sam Laidler,Z9YKSNFDZ0JDBTEHGXJS2K1P3G,zYNb7Ma2bsuDpmM3Hr7CdZdaddYZY
6,Corey Cleave,2,02 Sep,Old Tenisonians 2s,10,Corey Cleave,0HTAK3MQ28S69A5KBCTZ8HKHC0,5LvCuVXJi4SbtnSTDjWOrXiMBzaZY
7,Tom Riley,2,02 Sep,Old Tenisonians 2s,10,Tom Riley,HETKAQ4MN17GHFSVRBG687N2JG,jOc90auJoXLe0uBQ6KMAGkXC38LZY
8,Harold Kiyingi,2,02 Sep,Old Tenisonians 2s,10,Harold Kiyingi,K7PXE48P0H4Z5EW9ZMAZVKN6ZG,t5DNpFgjAqOrOXgdxf3sWc0LFrOZY
9,Joe Docherty,2,02 Sep,Old Tenisonians 2s,10,Joe Docherty,R4RHW9Q96H4C30FRJPTQ8TN25W,xrd6VU2RY4PPofA72IiDTNygU8YZY


# Create Invoices

In [143]:
invoices_api = client.invoices

In [144]:
invoices = []

for i, row in order_mg.iterrows():
    order_id = row['obj_id']
    cust_id = row['id']
    invoice_number = str(dt.datetime.now().date()) +f"_{row['OT Team']}"+f"_{row['obj_id']}_1"

    due_at =  str(dt.datetime.now().date() + dt.timedelta(days = 1))

    title = 'Match Subs ' + row['Game Date'] + ' vs ' + row['Opposition Name']

        
    body = {
    'invoice': {
        'location_id': 'SBBJSGR24YHYB',
        'order_id': f'{order_id}',
        'primary_recipient': {
            'customer_id': f'{cust_id}'
        },
        'payment_requests': [
            {
                'request_type': 'BALANCE',
                'due_date': due_at,
                'tipping_enabled': True,
                'automatic_payment_source': 'NONE',
                'reminders': [
                    {
                        'relative_scheduled_days': -1,
                        'message': 'Your Invoice is due'
                    }
                ]
            }
        ],
        'delivery_method': 'EMAIL',
        'invoice_number': invoice_number,
        'title': title,
        'description': title,
        'scheduled_at': due_at+'T10:00:00Z',
        'accepted_payment_methods': {
            'card': True,
            'square_gift_card': False,
            'bank_account': False,
            'buy_now_pay_later': False,
            'cash_app_pay': False
        },
        'sale_or_service_date': str(dt.datetime.now().date()),
        'store_payment_method_enabled': False
    },
    'idempotency_key': f'{uuid.uuid1()}'
    }

    result = invoices_api.create_invoice(body)
    print(result)
    invidid=result.body['invoice']['id']
    invoices.append(invidid)

<ApiResponse {"invoice":{"id":"inv:0-ChAImh-csCAng261EGwHqsxAEIMP","version":0,"location_id":"SBBJSGR24YHYB","order_id":"tfIQU8O9eVkQgGxvtGhFHnNUY7eZY","payment_requests":[{"uid":"769ce965-0392-460b-ad10-3f8b7ea8e600","request_type":"BALANCE","due_date":"2023-09-04","tipping_enabled":true,"computed_amount_money":{"amount":1000,"currency":"GBP"},"total_completed_amount_money":{"amount":0,"currency":"GBP"},"reminders":[{"uid":"e05d75bc-d725-487f-b568-2d7bd468c324","relative_scheduled_days":-1,"message":"Your Invoice is due","status":"PENDING"}],"automatic_payment_source":"NONE"}],"primary_recipient":{"customer_id":"T57NBJNMRHX79ZV68VW8SK166R","given_name":"Ade","family_name":"Lawal","email_address":"adelawal@live.co.uk","phone_number":"+447769116015"},"invoice_number":"2023-09-03_4_T57NBJNMRHX79ZV68VW8SK166R_1","title":"Match Subs 02 Sep vs Suttonians 4s","description":"Match Subs 02 Sep vs Suttonians 4s","scheduled_at":"2023-09-04T10:00:00Z","status":"DRAFT","timezone":"Europe/London","

In [145]:
for invidid in invoices:

    body = {
        'version': 0,
        'idempotency_key': f'{invidid}'
    }

    result = invoices_api.publish_invoice(
        invidid,
        body
    )
    print(result)

<ApiResponse {"invoice":{"id":"inv:0-ChAImh-csCAng261EGwHqsxAEIMP","version":1,"location_id":"SBBJSGR24YHYB","order_id":"tfIQU8O9eVkQgGxvtGhFHnNUY7eZY","payment_requests":[{"uid":"769ce965-0392-460b-ad10-3f8b7ea8e600","request_type":"BALANCE","due_date":"2023-09-04","tipping_enabled":true,"computed_amount_money":{"amount":1000,"currency":"GBP"},"total_completed_amount_money":{"amount":0,"currency":"GBP"},"reminders":[{"uid":"e05d75bc-d725-487f-b568-2d7bd468c324","relative_scheduled_days":-1,"message":"Your Invoice is due","status":"NOT_APPLICABLE"}],"automatic_payment_source":"NONE"}],"primary_recipient":{"customer_id":"T57NBJNMRHX79ZV68VW8SK166R","given_name":"Ade","family_name":"Lawal","email_address":"adelawal@live.co.uk","phone_number":"+447769116015"},"invoice_number":"2023-09-03_4_T57NBJNMRHX79ZV68VW8SK166R_1","title":"Match Subs 02 Sep vs Suttonians 4s","description":"Match Subs 02 Sep vs Suttonians 4s","scheduled_at":"2023-09-04T10:00:00Z","status":"SCHEDULED","timezone":"Europ